In [1]:
# 기본 작업 경로 설정

import os
notebook_path = os.path.abspath("project_3_git/readme.md")
notebook_dir = os.path.dirname(notebook_path)
os.chdir(notebook_dir)

# 현재 작업 디렉토리 출력
print("Current working directory: ", os.getcwd())

Current working directory:  /mnt/e/py_data/project_3_git


In [24]:
# json 파일 가져오기
import json
from sklearn.model_selection import train_test_split

# JSON 파일에서 딕셔너리 읽기
with open('data/text_data/output_text.json', 'r') as file:
    data_loaded = json.load(file)

In [26]:
all_of_data_sets = []
for i in data_loaded.values():
    all_of_data_sets.append(f'입력값 : {i[0]} \n출력값 : {i[1]}')

In [32]:
train_data, test_data = train_test_split(all_of_data_sets, train_size=5000, random_state=42)

In [35]:
len(train_data), len(test_data)

(5000, 994)

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments
from datasets import Dataset

def tokenize_function(examples):
    return tokenizer(examples, truncation=True, padding="max_length", max_length=128)

# Step 1: Tokenizer 및 모델 준비
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

In [53]:
tokenizer.pad_token = tokenizer.eos_token # 끝 토큰을 그냥 패드토큰으로 지정

# 문자열 리스트를 사전으로 변환 후 데이터셋 생성
train_dataset = Dataset.from_dict({"text": train_data})
test_dataset = Dataset.from_dict({"text": test_data})

# `text` 열의 데이터가 문자열 리스트 형태인지 확인합니다
train_dataset = train_dataset.map(tokenize_function, batched=True, remove_columns=["text"])
test_dataset = test_dataset.map(tokenize_function, batched=True, remove_columns=["text"])

# Data collator 준비
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

# Step 5: TrainingArguments 설정
training_args = TrainingArguments(
    output_dir='models/gpt2/gpt2_base_0/',
    overwrite_output_dir=True,
    num_train_epochs=100,
    per_device_train_batch_size=10,
    per_device_eval_batch_size=10,
    save_steps=100,
    save_total_limit=5,
    save_strategy='epoch',
    evaluation_strategy='epoch',
    load_best_model_at_end=True,
    report_to='tensorboard',  # TensorBoard로 로깅
    logging_dir='models/gpt2/gpt2_base_0/logs',  # 로그 파일이 저장될 디렉토리
    logging_steps=10,  # 로깅 간격
    remove_unused_columns=False 
)

# Step 6: Trainer 설정 및 학습 시작
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset  # 평가 데이터셋 추가
)

trainer.train()

Map: 100%|██████████████████████████████████████████████████████████████████████| 994/994 [00:00<00:00, 4857.08 examples/s]
/home/tommy/miniconda3/envs/p3/lib/python3.11/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.493300,0.450099
2,0.429600,0.379033
3,0.391400,0.346459
4,0.346800,0.329889
5,0.276200,0.320638
6,0.289000,0.311793
7,0.278500,0.310392


In [9]:
# Step 6: 모델 저장
model.save_pretrained('models/gpt2/gpt2_base_0/models')
tokenizer.save_pretrained('models/gpt2/gpt2_base_0/models/tokenizer')

('./fine_tuned_gpt2/tokenizer_config.json',
 './fine_tuned_gpt2/special_tokens_map.json',
 './fine_tuned_gpt2/vocab.json',
 './fine_tuned_gpt2/merges.txt',
 './fine_tuned_gpt2/added_tokens.json')

In [6]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# 저장된 모델 및 토크나이저 불러오기
model_path = './fine_tuned_gpt2'
model = GPT2LMHeadModel.from_pretrained(model_path)
tokenizer = GPT2Tokenizer.from_pretrained(model_path)

# 평가 모드로 변경
model.eval()

def generate_text(prompt, model, tokenizer, max_length=128, num_return_sequences=1):
    # 입력 텍스트를 토큰화
    inputs = tokenizer.encode(prompt, return_tensors='pt')

    # 생성 인자를 설정하여 모델이 텍스트를 생성
    outputs = model.generate(
        inputs,
        max_length=max_length,
        num_return_sequences=num_return_sequences,
        no_repeat_ngram_size=2,
        top_k=50,
        top_p=0.95,
        temperature=1.9,
        do_sample=True,
        early_stopping=True
    )

    # 생성된 텍스트를 디코딩
    generated_texts = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
    
    return generated_texts

# 예시: "prompt"에 원하는 문장을 넣어서 결과를 확인
prompt = "오늘"
generated_texts = generate_text(prompt, model, tokenizer)
    
for i, text in enumerate(generated_texts):
    print(f"Generated Text {i+1}:")
    print(text)
    print()

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Text 1:
오늘 하루도 사랑과 기쁨요.
잉상 밝게 진심으로 행복한 순간들을 만껏 좋은 일쳴세어 원합니다.

